Given that we do not appear to have access to the Minerva sims, I'm going to compute covariances from the test boxes. I'm gonna start by computing a separate one for each HOD and Cosmology used, across realizations, and convince myself they don't vary much. 

In [1]:
from pearce.mocks.kittens import TestBox
from pearce.mocks import tpcf_subregions
import numpy as np
from collections import OrderedDict
from time import time
from scipy.optimize import minimize_scalar

In [2]:
import matplotlib
#matplotlib.use('Agg')
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [3]:
nd = 1e-4
min_ptcl = 100
r_bins = np.logspace(-1.1, 1.6, 19)
logMmin_bounds = (12.0, 14.0)

In [4]:
hod_param_ranges =  OrderedDict({'sigma_logM': [0.05, 0.3], 'logM0': [13.1, 14.6], 'logM1': [13.1, 14.6], 'alpha': [0.8, 1.2]})

In [5]:
def make_LHC(ordered_params, N, seed = None):

    if seed is None:
        seed = int(time())
    np.random.seed(seed)

    points = []
    # by linspacing each parameter and shuffling, I ensure there is only one point in each row, in each dimension.
    for plow, phigh in ordered_params.itervalues():
        point = np.linspace(plow, phigh, num=N)
        np.random.shuffle(point)  # makes the cube random.
        points.append(point)
    return np.stack(points).T

In [6]:
def add_logMmin(hod_params, cat):

    hod_params['logMmin'] = 13.0 #initial guess
    #cat.populate(hod_params) #may be overkill, but will ensure params are written everywhere
    def func(logMmin, hod_params):
        hod_params.update({'logMmin':logMmin}) 
        return (cat.calc_analytic_nd(hod_params) - nd)**2

    res = minimize_scalar(func, bounds = logMmin_bounds, args = (hod_params,), options = {'maxiter':100}, method = 'Bounded')

    # assuming this doens't fail
    #print 'logMmin', res.x
    hod_params['logMmin'] = res.x

In [7]:
N = 10
LHC = make_LHC(hod_param_ranges, N)
hod_dicts = [dict(zip(hod_param_ranges.keys(), vals)) for vals in LHC]

In [13]:
def compute_full_jk(cat, rbins, rand_scalecut = 1.0,  n_rands= [5, 5], n_sub = 5):
    #np.random.seed(int(time()))
    
    n_cores = cat._check_cores(100)

    x_g, y_g, z_g = [cat.model.mock.galaxy_table[c] for c in ['x', 'y', 'z']]
    #pos_g = return_xyz_formatted_array(x_g, y_g, z_g, period=cat.Lbox)
    pos_g = np.vstack([x_g, y_g, z_g]).T

    x_m, y_m, z_m = [cat.halocat.ptcl_table[c] for c in ['x', 'y', 'z']]
    #pos_m = return_xyz_formatted_array(x_m, y_m, z_m, period=cat.Lbox)
    pos_m = np.vstack([x_m, y_m, z_m]).T

    rbins = np.array(rbins)
    rbins_small,rbins_large = list(rbins[rbins < rand_scalecut]), list(rbins[rbins >= rand_scalecut])

    rbins_large.insert(0, rbins_small[-1]) # make sure the middle bin is not cut

    xi_ggs, xi_gms, cov_ggs, cov_gms = [], [], [], []
    #for rb, nr in zip([rbins_small, rbins_large], n_rands): #
    for rb, nr in zip([rbins_large], n_rands): #

        randoms = np.random.random((pos_g.shape[0] * nr,
                                    3)) * cat.Lbox / cat.h  # Solution to NaNs: Just fuck me up with randoms
        xi_gg, xi_gm = tpcf_subregions(pos_g / cat.h, randoms, rb, sample2 = pos_m/cat.h, period=cat.Lbox / cat.h,
                                    num_threads=n_cores, Nsub=n_sub, estimator='Landy-Szalay',\
                                       do_auto1 = True, do_cross = True, do_auto2 = False)
        xi_ggs.append(xi_gg)
        xi_gms.append(xi_gm)

        cov_ggs.append(xi_gg_cov)
        cov_gms.append(xi_gm_cov)


    #xi_gg = np.hstack(np.hstack(xi_ggs))
    full_cov = cov_ggs
    full_cov.extend(cov_gms)
    cov = block_diag(*full_cov)
    return cov

In [ ]:
cov_mats = np.zeros((7,N, 5, 2*len(r_bins)-2, 2*len(r_bins)-2))

for boxno in xrange(7):
    for realization in xrange(5):
        cat = TestBox(boxno = boxno, realization = realization, system = 'sherlock')
        cat.load(1.0, HOD = str('zheng07'), particles = True, downsample_factor = 1e-2)
        for hod_idx, hod_params in enumerate(hod_dicts):
            add_logMmin(hod_params, cat)
            cat.populate(hod_params)
            compute_full_jk(cat, r_bins)
            break
        break
    break

/home/users/swmclau2/.local/lib/python2.7/site-packages/pearce/mocks/cat.py:284: UserWarning: n_cores invalid. Changing from 100 to maximum 16.
  warnings.warn('n_cores invalid. Changing from %d to maximum %d.' % (n_cores, max_cores))
